In [1]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 20.0 MB/s 
     |████████████████████████████████| 6.3 MB 41.2 MB/s 
     |████████████████████████████████| 16.7 MB 294 kB/s 


In [2]:
import fiona
from fiona.crs import from_epsg
from pyproj import CRS
import os
from matplotlib.patches import Ellipse
from shapely.geometry import Polygon, mapping
import numpy as np
from tqdm import tqdm
from osgeo import gdal

from google.colab import drive
drive.mount("/content/drive")
os.chdir('/content/drive/MyDrive/ASU - Zhiang/Projects/instance segmentation/')
os.listdir()

Mounted at /content/drive


['data',
 'instance_registration.ipynb',
 'TGRS',
 'instance_registration_IoU.ipynb',
 'random_ellipse_shapefile_generation.ipynb',
 'tile_splitting.ipynb',
 'instance_registration_IoU_all_adjacents.ipynb']

# generate random ellipses 

In [3]:
epsg = 32611  # WGS 84 / UTM zone 11N
crs = from_epsg(epsg)
print(CRS.from_epsg(epsg))

epsg:32611


In [4]:
def generate_ellipse(center_x, center_y, width, height, angle):
  ellipse = Ellipse((center_x, center_y), width, height, angle) 
  vertices = ellipse.get_verts()     # get the vertices from the ellipse object
  ellipse = Polygon(vertices)        # Turn it into a polygon
  return ellipse

In [24]:
def spawn_ellipses(tile_length, ellipse_length_range, density, tile_number):
  """
  tile: tile_length/tile_width in meters
  ellipse: length_length_range in meters
  density: average number of ellipses per tile
  study area: tile_number along north (the total study are is tile_length^2 * tile_number^2)
  tile origin is (0, 0)
  """
  ellipses = []
  N = tile_number**2 * density
  X = tile_length * tile_number
  Y = tile_length * tile_number
  center_x_list = np.random.rand(N) * X
  center_y_list = np.random.rand(N) * Y
  angle_list = np.random.rand(N) * 180
  width_list = np.random.uniform(ellipse_length_range[0], ellipse_length_range[1], N)
  height_list = np.random.uniform(ellipse_length_range[0], ellipse_length_range[1], N)
  for i in tqdm(range(N)):
    ellipse = generate_ellipse(center_x_list[i], center_y_list[i], width_list[i], height_list[i], angle_list[i])
    ellipses.append(ellipse)
  return ellipses

ellipse_max = 1.5
ellipse_min = 0.2
tile_length = 10 
ellipse_range = (ellipse_min, ellipse_max)
density = 400
tile_number = 3

ellipses = spawn_ellipses(tile_length, ellipse_range, density, tile_number)


100%|██████████| 3600/3600 [00:00<00:00, 6006.57it/s]


In [25]:
schema = {
    'geometry':'Polygon',
    'properties':[('id','str')]
}

# 2. open a write file
polyShp = fiona.open('data/random_generation/random_400_3.shp', mode='w', driver='ESRI Shapefile', schema = schema, crs = crs)

# 3. add polygons to file
for id, polygon in enumerate(ellipses):
  rowDict = {'geometry' : mapping(polygon), 'properties': {'id': str(id)}}
  polyShp.write(rowDict)

# 4. write file
polyShp.close()


# generate a fake tiff file

In [23]:
from osgeo import gdal
from osgeo import osr
import numpy as np
import os, sys

# raster resolution
xres = 0.01
yres = 0.01

#  Choose some Geographic Transform (Around Lake Tahoe)
lat = [0, tile_length*tile_number + ellipse_max/2]
lon = [0, tile_length*tile_number + ellipse_max/2]

#  Calculate the Image Size
image_size = (int(lat[1]/xres), int(lon[1]/yres))

#  Create Each Channel
r_pixels = np.ones((image_size), dtype=np.uint8)*0

# set geotransform
nx = image_size[0]
ny = image_size[1]
xmin, ymin, xmax, ymax = [min(lon), min(lat), max(lon), max(lat)]
geotransform = (xmin, xres, 0, ymax, 0, -yres)

# create the 3-band raster file
dst_ds = gdal.GetDriverByName('GTiff').Create('data/random_generation/random_400_3.tif', ny, nx, 1, gdal.GDT_Byte)

dst_ds.SetGeoTransform(geotransform)    # specify coords
srs = osr.SpatialReference()            # establish encoding
srs.ImportFromEPSG(epsg)                # WGS84 lat/long
dst_ds.SetProjection(srs.ExportToWkt()) # export coords to file
dst_ds.GetRasterBand(1).WriteArray(r_pixels)   # write r-band to the raster
dst_ds.FlushCache()                     # write to disk
dst_ds = None